for markdown basic syntax see [here](https://www.markdownguide.org/basic-syntax/).

# Nucleotide to amino acid sequence conversion, protein alignment and protein function prediction

**After calculating the desired SNPs for each sample**, the coding sequences need to be extracted, converted to protein sequences and these then need to be aligned in order to ascertain which SNPs are causing amino acid changes. VCF cleanup, SNP categorization and read support percentage calculation should be done beforehand.  
To do this, a **GFF3 file with reference annotation** is needed as well as the reference **genome.fasta** and a **consensus.fasta** for each sample at each desired SNP cutoff level.

**Tools**: gffread, blastp, local bash and ubuntu tools.

#### Inputs:
Don't put the final "/" on paths.  
Variables with "name" should contain just the name of the file and variables without it should contain the complete path to the file excluding the format.  
Arrays **cds_list** and **gene_csv** are redundant but should be kept as such.

In [ ]:
#desired folder for the output of this notebook
analysis_dir='/home/fuberlin/Documents/Runs/Illumina/20220421_MDV_HSV1_BAC/MDV'
mkdir "$analysis_dir"
cd "$analysis_dir"/

#if output folder does not exist one will be created
output_dir='/home/fuberlin/Documents/Runs/Illumina/20220421_MDV_HSV1_BAC/MDV/Protein_Prediction/output' 
mkdir "$output_dir"

#complete path to a ref FASTA file, and separate variable for the name of the reference
reference='/home/fuberlin/Documents/Runs/Illumina/20220421_MDV_HSV1_BAC/MDV/Protein_Prediction/MDV_ref'
refname='MDV_ref'

#annotation is a gff3 file that can be extracted directly from the ncbi accession page
annotation='/home/fuberlin/Documents/Runs/Illumina/20220421_MDV_HSV1_BAC/MDV/Protein_Prediction/MDV_ref'

#codingsequences is a txt file that can be extracted directly from the ncbi accession page, separate variable for the name of the codingsequences file
codingsequences='/home/fuberlin/Documents/Runs/Illumina/20220421_MDV_HSV1_BAC/MDV/Protein_Prediction/MDV_coding_sequences.txt'
csname='MDV_coding_sequences.txt'

#proteinsequences is a txt file that can be extracted directly from the ncbi accession page, separate variable for the name of the proteinsequences file
proteinsequences='/home/fuberlin/Documents/Runs/Illumina/20220421_MDV_HSV1_BAC/MDV/Protein_Prediction/MDV_protein_sequences.txt'
psname='MDV_protein_sequences.txt'

#samples is a list of the sample codes generated in sequencing, ending in _SXX
samples=(CEF-WT-PLX-2_S9 CEF-WT-PX-1_S1 CEF-WT-PX-2_S3 CEF-WT-PXXX-1_S5 CEF-WT-PXXX-2_S7 CEF-YS-PLX-2_S10 CEF-YS-PX-1_S2 CEF-YS-PX-2_S4 CEF-YS-PXXX-1_S6 CEF-YS-PXXX-2_S8 T7-WT-PXII_S11 T7-WT-PXX_S12)


#snp_dir is the folder where the SNP calculations were made (should contain 1 folder per sample of standard SNP outputs as per the following script: Variant_Calling_6modes_MN08022022.sh)
snp_dir='/home/fuberlin/Documents/Runs/Illumina/20220421_MDV_HSV1_BAC/MDV/SNP_Prediction'

### Reference (gene IDs and locations)
Getting a table of gene IDs and locations for labelling of SNPs later on.

In [ ]:
grep -e ">" "$codingsequences" > cs1.txt
sed -e 's/^.*gene/gene/g' -e 's/gene\=//g' -e 's/\].*\[location\=/\ /g' -e 's/\.\..*\.\./\.\./g' -e 's/\.\./\ /g' -e 's/complement(//g' -e 's/join(//g' -e 's/)\]//g' -e 's/\[gbkey\=CDS\]//g' -e 's/\.\./\ /g' -e 's/\]//g' -e 's/\ /,/' -e 's/\ /,/' -e 's/)//g' -e 's/>.*\=/>/g' -e 's/\,/\ /g' -e 's/>//g' -e 's/\ /\,/g' cs1.txt > cs.csv
echo "CSV generated"
gene_csv=( $(tail -n +1 cs.csv | cut -d ',' -f1) )
start_csv=( $(tail -n +1 cs.csv | cut -d ',' -f2) )
end_csv=( $(tail -n +1 cs.csv | cut -d ',' -f3) )
#Report
echo "Gene ID and location arrays generated."
echo ""${gene_csv[@]}""
echo ""${start_csv[@]}""
echo ""${end_csv[@]}""

### Reference (nt to aa)
Generating the protein repertoire from the reference, to align all the proteins from the sample consensus.

In [ ]:
#extraction of protein information using gffread
#gffread -C "$annotation".gff3 -g "$reference".fasta -y proteins.fasta
sed -e 's/^.*gene/>gene/g' -e 's/gene\=//g' -e 's/\].*\[location\=/\ /g' -e 's/\.\..*\.\./\.\./g' -e 's/\.\./\ /g' -e 's/complement(//g' -e 's/join(//g' -e 's/)\]//g' -e 's/\[gbkey\=CDS\]//g' -e 's/\.\./\ /g' -e 's/\]//g' -e 's/\ /,/' -e 's/\ /,/' -e 's/)//g' -e 's/>.*\=/>/g' -e 's/\,.*//g' -e 's/>/>gene\-/g' "$proteinsequences" > proteins.fasta
awk '/^>/ {$0=$0 "_ref"}1' proteins.fasta>protein.fasta
#create folder for protein fastas and move into it
mkdir "$output_dir"/"$refname"/
#move file to protein fastas folder
mv protein.fasta proteins.fasta "$output_dir"/"$refname"/
cd "$output_dir"/"$refname"/
cat protein.fasta | awk '{
        if (substr($0, 1, 1)==">") {filename=(substr($0,2) ".fasta")}
        print $0 >> filename
        close(filename)
}'
#Report
echo "Reference protein sequences generated."

### NT to AA (samples)

The headers of the consensus files must match the headers of the fasta reference and GFF3 file.

#### Importing the consensus files and renaming them
This may not be needed anymore if the VC scripts are updated the consensus files are renamed at the point of their creation.

In [ ]:
mkdir "$analysis_dir"/csensus/
cd "$snp_dir"
for s in "${samples[@]}"
do
#for 50% SNP read support
cd "$snp_dir"/"$s"/Variant_Calling_0.5_SNPs/
cp *.fa "$analysis_dir"/csensus/
cd "$analysis_dir"/csensus/
rename 's/_freebayes_0.5_SNPs.fa/_50.fasta/g' *.fa

#for 10% SNP read support
cd "$snp_dir"/"$s"/Variant_Calling_0.1_SNPs/
cp *.fa "$analysis_dir"/csensus/
cd "$analysis_dir"/csensus/
rename 's/_freebayes_0.1_SNPs.fa/_10.fasta/g' *.fa

#for 5% SNP read support
cd "$snp_dir"/"$s"/Variant_Calling_0.05_SNPs/
cp *.fa "$analysis_dir"/csensus/
cd "$analysis_dir"/csensus/
rename 's/_freebayes_0.05_SNPs.fa/_5.fasta/g' *.fa

#for 1% SNP read support
cd "$snp_dir"/"$s"/Variant_Calling_0.01_SNPs/
cp *.fa "$analysis_dir"/csensus/
cd "$analysis_dir"/csensus/
rename 's/_freebayes_0.01_SNPs.fa/_1.fasta/g' *.fa

#Report
echo "Consensus file for "$s" re-formatted."
done

For HSV1 Pol Mutant data, the reference was generated by using sequencing data generated in house to calculate SNPs and apply them to the Strain F reference. This was done because the stocks used for this work should already be a few nucleotides away from the published Strain F reference.  
  
**ENTER FASTA HEADER HERE**

In [ ]:
#to match the headers of the consensus files with the header of the reference, shouldn't be needed in other runs
for fasta in $(find "$analysis_dir"/csensus/ -name "*.fasta")
do
    new_fasta=$(basename $fasta .fasta).new.fasta
    sed -e 's/>.*/>EF523390.1/g' $fasta > $new_fasta
done
mkdir ./corrected/
mv *.new.fasta ./corrected/
rm *.fasta
cd ./corrected/
mv *.fasta "$analysis_dir"/csensus/
cd "$analysis_dir"/csensus/
rm -r ./corrected/
rename 's/.new.fasta/.fasta/g' *.fasta

#### Getting the protein sequences
Generating the CDS/protein repertoire from each sample at 50% and 10% cutoff. 

In [ ]:
cd "$analysis_dir"/
for s in "${samples[@]}"
do

#for 50% SNP read support
gffread "$annotation".gff3 -g "$analysis_dir"/csensus/consensus_"$s"_50.fasta -y pt_"$s"_50.fasta
#add sample to the end of each title line ">"
awk '/^>/ {$0=$0 "_'"$s"'"}1' pt_"$s"_50.fasta>protein_"$s"_50.fasta
mkdir "$output_dir"/"$s"_50/
mv pt_"$s"_50.fasta protein_"$s"_50.fasta "$output_dir"/"$s"_50/
cd "$output_dir"/"$s"_50/
#separating the multifasta into individual fastas named after the lines with ">"
cat protein_"$s"_50.fasta | awk '{
        if (substr($0, 1, 1)==">") {filename=(substr($0,2) ".fasta")}
        print $0 >> filename
        close(filename)
}'

#for 10% SNP read support
gffread "$annotation".gff3 -g "$analysis_dir"/csensus/consensus_"$s"_10.fasta -y pt_"$s"_10.fasta
#add sample to the end of each title line ">"
awk '/^>/ {$0=$0 "_'"$s"'"}1' pt_"$s"_10.fasta>protein_"$s"_10.fasta
mkdir "$output_dir"/"$s"_10/
mv pt_"$s"_10.fasta protein_"$s"_10.fasta "$output_dir"/"$s"_10/
cd "$output_dir"/"$s"_10/
#separating the multifasta into individual fastas named after the lines with ">"
cat protein_"$s"_10.fasta | awk '{
        if (substr($0, 1, 1)==">") {filename=(substr($0,2) ".fasta")}
        print $0 >> filename
        close(filename)
}'

#for 5% SNP read support
gffread "$annotation".gff3 -g "$analysis_dir"/csensus/consensus_"$s"_5.fasta -y pt_"$s"_5.fasta
#add sample to the end of each title line ">"
awk '/^>/ {$0=$0 "_'"$s"'"}1' pt_"$s"_5.fasta>protein_"$s"_5.fasta
mkdir "$output_dir"/"$s"_5/
mv pt_"$s"_5.fasta protein_"$s"_5.fasta "$output_dir"/"$s"_5/
cd "$output_dir"/"$s"_5/
#separating the multifasta into individual fastas named after the lines with ">"
cat protein_"$s"_5.fasta | awk '{
        if (substr($0, 1, 1)==">") {filename=(substr($0,2) ".fasta")}
        print $0 >> filename
        close(filename)
}'

#for 1% SNP read support
gffread "$annotation".gff3 -g "$analysis_dir"/csensus/consensus_"$s"_1.fasta -y pt_"$s"_1.fasta
#add sample to the end of each title line ">"
awk '/^>/ {$0=$0 "_'"$s"'"}1' pt_"$s"_1.fasta>protein_"$s"_1.fasta
mkdir "$output_dir"/"$s"_1/
mv pt_"$s"_1.fasta protein_"$s"_1.fasta "$output_dir"/"$s"_1/
cd "$output_dir"/"$s"_1/
#separating the multifasta into individual fastas named after the lines with ">"
cat protein_"$s"_1.fasta | awk '{
        if (substr($0, 1, 1)==">") {filename=(substr($0,2) ".fasta")}
        print $0 >> filename
        close(filename)
}'

#Report
echo ""$s" protein sequences generated."

done

### Aligning protein sequences
Getting the corresponding proteins from the reference and sample repertoires and aligning them. To do this, we need to iterate through the list of samples but also through a list of proteins that is generated using the full protein repertoire of the reference before it is separated into ssFASTA files.

In [ ]:
#grabbing a list of all CDS/proteins available from the seq
cd "$output_dir"/"$refname"/
grep -e ">" proteins.fasta >list1.txt
awk 'sub(/^>/, "")' list1.txt >list2.txt 
rm list1.txt
readarray -t cds_list < list2.txt 
echo "${cds_list[@]}"

In [ ]:
#Protein alignments for 50% SNP support sequences
for s in "${samples[@]}"
do
    mkdir "$output_dir"/"$s"_50/alignments/
    cd "$output_dir"/"$s"_50/alignments/
    for cds in "${cds_list[@]}"
    do
    #When not provided, the default value is: 'qaccver saccver pident length mismatch gapopen qstart qend sstart send evalue bitscore', which is equivalent to the keyword 'std'
        sed -i -e '2,$s/\.//g' "$output_dir"/"$s"_50/"$cds"_"$s".fasta
        blastp -query "$output_dir"/"$s"_50/"$cds"_"$s".fasta -subject "$output_dir"/"$refname"/"$cds"_ref.fasta -out a_"$cds"_"$s".txt -outfmt 10
        tr , '\n' < a_"$cds"_"$s".txt > tmp_"$cds"_"$s".txt
        rm a_"$cds"_"$s".txt
        readarray -t a < tmp_"$cds"_"$s".txt
        if [[ "${a[4]}" == 0 ]]
        then
            rm tmp_"$cds"_"$s".txt
        elif [[ "${a[4]}" > 0 ]]
        then
            blastp -query "$output_dir"/"$s"_50/"$cds"_"$s".fasta -subject "$output_dir"/"$refname"/"$cds"_ref.fasta -out alignment_"$cds"_"$s"_50.txt -outfmt 5
            rm tmp_"$cds"_"$s".txt
        else
           echo "error on "$s": "$cds"."
        fi
    done
done
#Report
echo "Protein alignments for sequences at 50% SNP read support done."

#Protein alignment for 10% SNP support sequences
for s in "${samples[@]}"
do
    mkdir "$output_dir"/"$s"_10/alignments/
    cd "$output_dir"/"$s"_10/alignments/
    for cds in "${cds_list[@]}"
    do
    #When not provided, the default value is: 'qaccver saccver pident length mismatch gapopen qstart qend sstart send evalue bitscore', which is equivalent to the keyword 'std'
        sed -i -e '2,$s/\.//g' "$output_dir"/"$s"_10/"$cds"_"$s".fasta
        blastp -query "$output_dir"/"$s"_10/"$cds"_"$s".fasta -subject "$output_dir"/"$refname"/"$cds"_ref.fasta -out a_"$cds"_"$s".txt -outfmt 10
        tr , '\n' < a_"$cds"_"$s".txt > tmp_"$cds"_"$s".txt
        rm a_"$cds"_"$s".txt
        readarray -t a < tmp_"$cds"_"$s".txt
        if [[ "${a[4]}" == 0 ]]
        then
            rm tmp_"$cds"_"$s".txt
        elif [[ "${a[4]}" > 0 ]]
        then
            blastp -query "$output_dir"/"$s"_10/"$cds"_"$s".fasta -subject "$output_dir"/"$refname"/"$cds"_ref.fasta -out alignment_"$cds"_"$s"_10.txt -outfmt 5
            rm tmp_"$cds"_"$s".txt
        else
            echo "error on "$s": "$cds"."
        fi
    done
done
#Report
echo "Protein alignments for sequences at 10% SNP read support done."

#Protein alignments for 5% SNP support sequences
for s in "${samples[@]}"
do
    mkdir "$output_dir"/"$s"_5/alignments/
    cd "$output_dir"/"$s"_5/alignments/
    for cds in "${cds_list[@]}"
    do
    #When not provided, the default value is: 'qaccver saccver pident length mismatch gapopen qstart qend sstart send evalue bitscore', which is equivalent to the keyword 'std'
        sed -i -e '2,$s/\.//g' "$output_dir"/"$s"_5/"$cds"_"$s".fasta
        blastp -query "$output_dir"/"$s"_5/"$cds"_"$s".fasta -subject "$output_dir"/"$refname"/"$cds"_ref.fasta -out a_"$cds"_"$s".txt -outfmt 10
        tr , '\n' < a_"$cds"_"$s".txt > tmp_"$cds"_"$s".txt
        rm a_"$cds"_"$s".txt
        readarray -t a < tmp_"$cds"_"$s".txt
        if [[ "${a[4]}" == 0 ]]
        then
            rm tmp_"$cds"_"$s".txt
        elif [[ "${a[4]}" > 0 ]]
        then
            blastp -query "$output_dir"/"$s"_5/"$cds"_"$s".fasta -subject "$output_dir"/"$refname"/"$cds"_ref.fasta -out alignment_"$cds"_"$s"_5.txt -outfmt 5
            rm tmp_"$cds"_"$s".txt
        else
           echo "error on "$s": "$cds"."
        fi
    done
done
#Report
echo "Protein alignments for sequences at 5% SNP read support done."

#Protein alignment for 1% SNP support sequences
for s in "${samples[@]}"
do
    mkdir "$output_dir"/"$s"_1/alignments/
    cd "$output_dir"/"$s"_1/alignments/
    for cds in "${cds_list[@]}"
    do
    #When not provided, the default value is: 'qaccver saccver pident length mismatch gapopen qstart qend sstart send evalue bitscore', which is equivalent to the keyword 'std'
        sed -i -e '2,$s/\.//g' "$output_dir"/"$s"_1/"$cds"_"$s".fasta
        blastp -query "$output_dir"/"$s"_1/"$cds"_"$s".fasta -subject "$output_dir"/"$refname"/"$cds"_ref.fasta -out a_"$cds"_"$s".txt -outfmt 10
        tr , '\n' < a_"$cds"_"$s".txt > tmp_"$cds"_"$s".txt
        rm a_"$cds"_"$s".txt
        readarray -t a < tmp_"$cds"_"$s".txt
        if [[ "${a[4]}" == 0 ]]
        then
            rm tmp_"$cds"_"$s".txt
        elif [[ "${a[4]}" > 0 ]]
        then
            blastp -query "$output_dir"/"$s"_1/"$cds"_"$s".fasta -subject "$output_dir"/"$refname"/"$cds"_ref.fasta -out alignment_"$cds"_"$s"_1.txt -outfmt 5
            rm tmp_"$cds"_"$s".txt
        else
            echo "error on "$s": "$cds"."
        fi
    done
done
#Report
echo "Protein alignments for sequences at 1% SNP read support done."

In [ ]:
#Generating CSV files for AA changes in each sample at 50% SNP read support
for s in "${samples[@]}"
do
    cd "$output_dir"/"$s"_50/alignments/
    mkdir "$output_dir"/"$s"_50/working/
    mkdir "$output_dir"/"$s"_50/aa_changes/
    echo "Gene, Start, End, Possible SNP Start (Direct), Possible SNP End (Direct), Possible SNP Start (Complement), Possible SNP End (Complement), AA Change, PROVEAN Score, PROVEAN Prediction" > "$output_dir"/"$s"_50/aa_changes/"$s"_aa_change_50.csv
    counter=0
    for filename in *
    do
    counter=$((counter+1))
    cp "$filename" "$output_dir"/"$s"_50/working/
    cd "$output_dir"/"$s"_50/working/
    sed '1,48d' "$filename" > a_"$counter".txt
    sed '4,22d' a_"$counter".txt > b_"$counter".txt
    sed -e 's/\<Hsp_qseq\>//g' -e 's;\<\/\-Hsp\_qseq\>;;g' -e 's/\<Hsp_hseq\>//g' -e 's/\<\/\-Hsp\_hseq\>//g' -e 's/\<Hsp_midline\>//g' -e 's/\<\/\-Hsp_midline\>//g' b_"$counter".txt > c_"$counter".txt
    cat c_"$counter".txt | tr -d "[:blank:]" > d_"$counter".txt
    sed 's/^.\{,2\}//' d_"$counter".txt > e_"$counter".txt
    sed 's/...$//' e_"$counter".txt > f_"$counter".txt
    sed -n 1p f_"$counter".txt > text1_"$counter".txt
    sed -n 2p f_"$counter".txt > text2_"$counter".txt
    sed 's/./&\n/g' text1_"$counter".txt > t1_"$counter".txt
    sed 's/./&\n/g' text2_"$counter".txt > t2_"$counter".txt
    readarray -t query < t1_"$counter".txt
    readarray -t subj < t2_"$counter".txt
    #substituting AAs that are the same in reference and query arrays for dots
        for a in "${!query[@]}" 
        do
            if [[ "${query[$a]}" == "${subj[$a]}" ]]
            then
                query[$a]="."
            fi
        done
    #printing info about unique (non-dot) AAs in the query array into aa_changes.txt in the format: alignmentfile: AAinref|index+1|AAinquery, in which index+1 represents the AA position
        for b in "${!query[@]}"
        do
            if [[ ${query[$b]} != "." ]]
            then
                i=$((b+1))
                for gene in "${gene_csv[@]}"
                do
                    if [[ "$filename" =~ .*"$gene"_.* ]]
                    then
                        for j in "${!gene_csv[@]}"
                        do
                            if [[ "${gene_csv[$j]}" == "$gene" ]];
                            then
                            st_fr=$((start_csv[$j]+("$i"*3)-3))
                            end_fr=$((start_csv[$j]+("$i"*3)+3))
                            st_cmp=$((end_csv[$j]-("$i"*3)-3))
                            end_cmp=$((end_csv[$j]-("$i"*3)+3))
                            echo ""$gene","${start_csv[$j]}","${end_csv[$j]}","$st_fr","$end_fr","$st_cmp","$end_cmp","${subj[$b]}""$i""${query[$b]}"" >> "$output_dir"/"$s"_50/aa_changes/"$s"_aa_change_50.csv
                            #Creating the var files necessay for Provean analysis in the next steps
                            cd "$output_dir"/"$s"_50/aa_changes/
                            echo "" > "$gene".var
                            echo ""${subj[$b]}""$i""${query[$b]}"" >> "$gene".var
                            cd "$output_dir"/"$s"_50/working/
                            fi
                        done
                    fi
                done
            fi
        done
    #change back to original folder so that the loop won't break
    cd "$output_dir"/"$s"_50/alignments/
    done
rm -r "$output_dir"/"$s"_50/working/
#Report
echo "Amino acid change CSV file generated for sample "$s" at 50% SNP read support."
done

In [ ]:
#Generating CSV files for AA changes in each sample at 10% SNP read support
for s in "${samples[@]}"
do
    cd "$output_dir"/"$s"_10/alignments/
    mkdir "$output_dir"/"$s"_10/working/
    mkdir "$output_dir"/"$s"_10/aa_changes/
    echo "Gene, Start, End, Possible SNP Start (Direct), Possible SNP End (Direct), Possible SNP Start (Complement), Possible SNP End (Complement), AA Change, PROVEAN Score, PROVEAN Prediction" > "$output_dir"/"$s"_10/aa_changes/"$s"_aa_change_10.csv
    counter=0
    for filename in *
    do
    counter=$((counter+1))
    cp "$filename" "$output_dir"/"$s"_10/working/
    cd "$output_dir"/"$s"_10/working/
    sed '1,48d' "$filename" > a_"$counter".txt
    sed '4,22d' a_"$counter".txt > b_"$counter".txt
    sed -e 's/\<Hsp_qseq\>//g' -e 's;\<\/\-Hsp\_qseq\>;;g' -e 's/\<Hsp_hseq\>//g' -e 's/\<\/\-Hsp\_hseq\>//g' -e 's/\<Hsp_midline\>//g' -e 's/\<\/\-Hsp_midline\>//g' b_"$counter".txt > c_"$counter".txt
    cat c_"$counter".txt | tr -d "[:blank:]" > d_"$counter".txt
    sed 's/^.\{,2\}//' d_"$counter".txt > e_"$counter".txt
    sed 's/...$//' e_"$counter".txt > f_"$counter".txt
    sed -n 1p f_"$counter".txt > text1_"$counter".txt
    sed -n 2p f_"$counter".txt > text2_"$counter".txt
    sed 's/./&\n/g' text1_"$counter".txt > t1_"$counter".txt
    sed 's/./&\n/g' text2_"$counter".txt > t2_"$counter".txt
    readarray -t query < t1_"$counter".txt
    readarray -t subj < t2_"$counter".txt
    #substituting AAs that are the same in reference and query arrays for dots
        for a in "${!query[@]}" 
        do
            if [[ "${query[$a]}" == "${subj[$a]}" ]]
            then
                query[$a]="."
            fi
        done
    #printing info about unique (non-dot) AAs in the query array into aa_changes.txt in the format: alignmentfile: AAinref|index+1|AAinquery, in which index+1 represents the AA position
        for b in "${!query[@]}"
        do
            if [[ ${query[$b]} != "." ]]
            then
                i=$((b+1))
                for gene in "${gene_csv[@]}"
                do
                    if [[ "$filename" =~ .*"$gene"_.* ]]
                    then
                        for j in "${!gene_csv[@]}"
                        do
                            if [[ "${gene_csv[$j]}" == "$gene" ]];
                            then
                            st_fr=$((start_csv[$j]+("$i"*3)-3))
                            end_fr=$((start_csv[$j]+("$i"*3)+3))
                            st_cmp=$((end_csv[$j]-("$i"*3)-3))
                            end_cmp=$((end_csv[$j]-("$i"*3)+3))
                            echo ""$gene","${start_csv[$j]}","${end_csv[$j]}","$st_fr","$end_fr","$st_cmp","$end_cmp","${subj[$b]}""$i""${query[$b]}"" >> "$output_dir"/"$s"_10/aa_changes/"$s"_aa_change_10.csv
                            #Creating the var files necessay for Provean analysis in the next steps
                            cd "$output_dir"/"$s"_10/aa_changes/
                            echo "" > "$gene".var
                            echo ""${subj[$b]}""$i""${query[$b]}"" >> "$gene".var
                            cd "$output_dir"/"$s"_10/working/
                            fi
                        done
                    fi
                done
            fi
        done
    #change back to original folder so that the loop won't break
    cd "$output_dir"/"$s"_10/alignments/
    done
rm -r "$output_dir"/"$s"_10/working/
#Report
echo "Amino acid change CSV file generated for sample "$s" at 10% SNP read support."
done

In [ ]:
#Generating CSV files for AA changes in each sample at 5% SNP read support
for s in "${samples[@]}"
do
    cd "$output_dir"/"$s"_5/alignments/
    mkdir "$output_dir"/"$s"_5/working/
    mkdir "$output_dir"/"$s"_5/aa_changes/
    echo "Gene, Start, End, Possible SNP Start (Direct), Possible SNP End (Direct), Possible SNP Start (Complement), Possible SNP End (Complement), AA Change, PROVEAN Score, PROVEAN Prediction" > "$output_dir"/"$s"_5/aa_changes/"$s"_aa_change_5.csv
    counter=0
    for filename in *
    do
    counter=$((counter+1))
    cp "$filename" "$output_dir"/"$s"_5/working/
    cd "$output_dir"/"$s"_5/working/
    sed '1,48d' "$filename" > a_"$counter".txt
    sed '4,22d' a_"$counter".txt > b_"$counter".txt
    sed -e 's/\<Hsp_qseq\>//g' -e 's;\<\/\-Hsp\_qseq\>;;g' -e 's/\<Hsp_hseq\>//g' -e 's/\<\/\-Hsp\_hseq\>//g' -e 's/\<Hsp_midline\>//g' -e 's/\<\/\-Hsp_midline\>//g' b_"$counter".txt > c_"$counter".txt
    cat c_"$counter".txt | tr -d "[:blank:]" > d_"$counter".txt
    sed 's/^.\{,2\}//' d_"$counter".txt > e_"$counter".txt
    sed 's/...$//' e_"$counter".txt > f_"$counter".txt
    sed -n 1p f_"$counter".txt > text1_"$counter".txt
    sed -n 2p f_"$counter".txt > text2_"$counter".txt
    sed 's/./&\n/g' text1_"$counter".txt > t1_"$counter".txt
    sed 's/./&\n/g' text2_"$counter".txt > t2_"$counter".txt
    readarray -t query < t1_"$counter".txt
    readarray -t subj < t2_"$counter".txt
    #substituting AAs that are the same in reference and query arrays for dots
        for a in "${!query[@]}" 
        do
            if [[ "${query[$a]}" == "${subj[$a]}" ]]
            then
                query[$a]="."
            fi
        done
    #printing info about unique (non-dot) AAs in the query array into aa_changes.txt in the format: alignmentfile: AAinref|index+1|AAinquery, in which index+1 represents the AA position
        for b in "${!query[@]}"
        do
            if [[ ${query[$b]} != "." ]]
            then
                i=$((b+1))
                for gene in "${gene_csv[@]}"
                do
                    if [[ "$filename" =~ .*"$gene"_.* ]]
                    then
                        for j in "${!gene_csv[@]}"
                        do
                            if [[ "${gene_csv[$j]}" == "$gene" ]];
                            then
                            st_fr=$((start_csv[$j]+("$i"*3)-3))
                            end_fr=$((start_csv[$j]+("$i"*3)+3))
                            st_cmp=$((end_csv[$j]-("$i"*3)-3))
                            end_cmp=$((end_csv[$j]-("$i"*3)+3))
                            echo ""$gene","${start_csv[$j]}","${end_csv[$j]}","$st_fr","$end_fr","$st_cmp","$end_cmp","${subj[$b]}""$i""${query[$b]}"" >> "$output_dir"/"$s"_5/aa_changes/"$s"_aa_change_5.csv
                            #Creating the var files necessay for Provean analysis in the next steps
                            cd "$output_dir"/"$s"_5/aa_changes/
                            echo "" > "$gene".var
                            echo ""${subj[$b]}""$i""${query[$b]}"" >> "$gene".var
                            cd "$output_dir"/"$s"_5/working/
                            fi
                        done
                    fi
                done
            fi
        done
    #change back to original folder so that the loop won't break
    cd "$output_dir"/"$s"_5/alignments/
    done
rm -r "$output_dir"/"$s"_5/working/
#Report
echo "Amino acid change CSV file generated for sample "$s" at 5% SNP read support."
done

In [ ]:
#Generating CSV files for AA changes in each sample at 1% SNP read support
for s in "${samples[@]}"
do
    cd "$output_dir"/"$s"_1/alignments/
    mkdir "$output_dir"/"$s"_1/working/
    mkdir "$output_dir"/"$s"_1/aa_changes/
    echo "Gene, Start, End, Possible SNP Start (Direct), Possible SNP End (Direct), Possible SNP Start (Complement), Possible SNP End (Complement), AA Change, PROVEAN Score, PROVEAN Prediction" > "$output_dir"/"$s"_1/aa_changes/"$s"_aa_change_1.csv
    counter=0
    for filename in *
    do
    counter=$((counter+1))
    cp "$filename" "$output_dir"/"$s"_1/working/
    cd "$output_dir"/"$s"_1/working/
    sed '1,48d' "$filename" > a_"$counter".txt
    sed '4,22d' a_"$counter".txt > b_"$counter".txt
    sed -e 's/\<Hsp_qseq\>//g' -e 's;\<\/\-Hsp\_qseq\>;;g' -e 's/\<Hsp_hseq\>//g' -e 's/\<\/\-Hsp\_hseq\>//g' -e 's/\<Hsp_midline\>//g' -e 's/\<\/\-Hsp_midline\>//g' b_"$counter".txt > c_"$counter".txt
    cat c_"$counter".txt | tr -d "[:blank:]" > d_"$counter".txt
    sed 's/^.\{,2\}//' d_"$counter".txt > e_"$counter".txt
    sed 's/...$//' e_"$counter".txt > f_"$counter".txt
    sed -n 1p f_"$counter".txt > text1_"$counter".txt
    sed -n 2p f_"$counter".txt > text2_"$counter".txt
    sed 's/./&\n/g' text1_"$counter".txt > t1_"$counter".txt
    sed 's/./&\n/g' text2_"$counter".txt > t2_"$counter".txt
    readarray -t query < t1_"$counter".txt
    readarray -t subj < t2_"$counter".txt
    #substituting AAs that are the same in reference and query arrays for dots
        for a in "${!query[@]}" 
        do
            if [[ "${query[$a]}" == "${subj[$a]}" ]]
            then
                query[$a]="."
            fi
        done
    #printing info about unique (non-dot) AAs in the query array into aa_changes.txt in the format: alignmentfile: AAinref|index+1|AAinquery, in which index+1 represents the AA position
        for b in "${!query[@]}"
        do
            if [[ ${query[$b]} != "." ]]
            then
                i=$((b+1))
                for gene in "${gene_csv[@]}"
                do
                    if [[ "$filename" =~ .*"$gene"_.* ]]
                    then
                        for j in "${!gene_csv[@]}"
                        do
                            if [[ "${gene_csv[$j]}" == "$gene" ]];
                            then
                            st_fr=$((start_csv[$j]+("$i"*3)-3))
                            end_fr=$((start_csv[$j]+("$i"*3)+3))
                            st_cmp=$((end_csv[$j]-("$i"*3)-3))
                            end_cmp=$((end_csv[$j]-("$i"*3)+3))
                            echo ""$gene","${start_csv[$j]}","${end_csv[$j]}","$st_fr","$end_fr","$st_cmp","$end_cmp","${subj[$b]}""$i""${query[$b]}"" >> "$output_dir"/"$s"_1/aa_changes/"$s"_aa_change_1.csv
                            #Creating the var files necessay for Provean analysis in the next steps
                            cd "$output_dir"/"$s"_1/aa_changes/
                            echo "" > "$gene".var
                            echo ""${subj[$b]}""$i""${query[$b]}"" >> "$gene".var
                            cd "$output_dir"/"$s"_1/working/
                            fi
                        done
                    fi
                done
            fi
        done
    #change back to original folder so that the loop won't break
    cd "$output_dir"/"$s"_1/alignments/
    done
rm -r "$output_dir"/"$s"_1/working/
#Report
echo "Amino acid change CSV file generated for sample "$s" at 1% SNP read support."
done

In [ ]:
#Housekeeping, removing all unecessary files
cd "$analysis_dir"/
mkdir results
for s in "${samples[@]}"
do
mkdir -p results/"$s"/aa_changes/
mv "$output_dir"/"$s"_50/aa_changes/"$s"_aa_change_50.csv "$analysis_dir"/results/"$s"/aa_changes/
mv "$output_dir"/"$s"_10/aa_changes/"$s"_aa_change_10.csv "$analysis_dir"/results/"$s"/aa_changes/
mv "$output_dir"/"$s"_5/aa_changes/"$s"_aa_change_5.csv "$analysis_dir"/results/"$s"/aa_changes/
mv "$output_dir"/"$s"_1/aa_changes/"$s"_aa_change_1.csv "$analysis_dir"/results/"$s"/aa_changes/
done

## VCF File Formatting

In [ ]:
#VCF Formatting for 50% SNP read support
for s in "${samples[@]}"
do 
cd "$snp_dir"/"$s"/"$s"_Export/
sed '1,60d' "$snp_dir"/"$s"/"$s"_Export/bwa_paired_"$s"_sorted_freebayes_0.5_SNPs.vcf > 1_50.vcf
sed -e "s/AB\=.*:GL//g" -e "s/.\/.\://g" -e "s/\:/\,/1" -e "s/\:.*//g" -e "s/\s\+/,/g" -e "s/\,ID//g" -e "s/\,FILTER\,INFO\,FORMAT\,unknown/\,DEPTH\,REF\_READS\,ALT\_READS\,TYPE/g" -e "s/\,\.//g" -e "s/\#CHROM\,//g" -e "s/\,/\:/1" -e "s/.*\://g" -e "s/REF/POS\,REF/g" 1_50.vcf > 2_50.vcf
echo "POS,REF,ALT,QUAL,DEPTH,REF_READS,ALT_READS,TYPE" > 3_50.vcf
    while IFS= read -r line; 
    do
        if [[ "$line" =~ ",A," ]] && [[ "$line" =~ ",C," ]]
        then
           echo ""$line",transversion" >> 3_50.vcf
        elif [[ "$line" =~ ",A," ]] && [[ "$line" =~ ",G," ]]
        then
            echo ""$line",transition" >> 3_50.vcf
        elif [[ "$line" =~ ",A," ]] && [[ "$line" =~ ",T," ]]
        then
            echo ""$line",transversion" >> 3_50.vcf
        elif [[ "$line" =~ ",C," ]] && [[ "$line" =~ ",G," ]]
        then
            echo ""$line",transversion" >> 3_50.vcf
        elif [[ "$line" =~ ",C," ]] && [[ "$line" =~ ",T," ]]
        then
            echo ""$line",transition" >> 3_50.vcf
        elif [[ "$line" =~ ",G," ]] && [[ "$line" =~ ",T," ]]
        then
            echo ""$line",transversion" >> 3_50.vcf
        else
            echo ""$line",substitution" >> 3_50.vcf    
        fi
    done < 2_50.vcf
sed '2d' 3_50.vcf  > 4_50.vcf
sed -i '1d' 4_50.vcf
echo "POS,REF,ALT,QUAL,DEPTH,REF_READS,ALT_READS,TYPE,GENE" > "$s"_SNPs_50.vcf
    while IFS= read -r line; 
    do
    pos=$(echo ""$line"" | sed  's/,.*$//');
    nc_region='TRUE'
        for c in "${!gene_csv[@]}"
        do
            if (( "$pos" >= "${start_csv[$c]}" && "$pos" <= "${end_csv[$c]}" ));
            then
                nc_region='FALSE'
                echo ""$line",""${gene_csv[$c]}""," >> "$s"_SNPs_50.vcf;
            fi
        done
        if [[ "$nc_region" == 'TRUE' ]];
        then 
            echo ""$line",-," >> "$s"_SNPs_50.vcf;
        fi
        nc_region='TRUE'
    done < 4_50.vcf
rm 1_50.vcf
rm 2_50.vcf
rm 3_50.vcf
rm 4_50.vcf
done

#VCF Formatting for 10% SNP read support
for s in "${samples[@]}"
do
cd "$snp_dir"/"$s"/"$s"_Export/
sed '1,60d' "$snp_dir"/"$s"/"$s"_Export/bwa_paired_"$s"_sorted_freebayes_0.1_SNPs.vcf > 1_10.vcf
sed -e "s/AB\=.*:GL//g" -e "s/.\/.\://g" -e "s/\:/\,/1" -e "s/\:.*//g" -e "s/\s\+/,/g" -e "s/\,ID//g" -e "s/\,FILTER\,INFO\,FORMAT\,unknown/\,DEPTH\,REF\_READS\,ALT\_READS\,TYPE/g" -e "s/\,\.//g" -e "s/\#CHROM\,//g" -e "s/\,/\:/1" -e "s/.*\://g" -e "s/REF/POS\,REF/g" 1_10.vcf > 2_10.vcf
echo "POS,REF,ALT,QUAL,DEPTH,REF_READS,ALT_READS,TYPE,GENE" > 3_10.vcf
    while IFS= read -r line; 
    do
        if [[ "$line" =~ ",A," ]] && [[ "$line" =~ ",C," ]]
        then
           echo ""$line",transversion" >> 3_10.vcf
        elif [[ "$line" =~ ",A," ]] && [[ "$line" =~ ",G," ]]
        then
            echo ""$line",transition" >> 3_10.vcf
        elif [[ "$line" =~ ",A," ]] && [[ "$line" =~ ",T," ]]
        then
            echo ""$line",transversion" >> 3_10.vcf
        elif [[ "$line" =~ ",C," ]] && [[ "$line" =~ ",G," ]]
        then
            echo ""$line",transversion" >> 3_10.vcf
        elif [[ "$line" =~ ",C," ]] && [[ "$line" =~ ",T," ]]
        then
            echo ""$line",transition" >> 3_10.vcf
        elif [[ "$line" =~ ",G," ]] && [[ "$line" =~ ",T," ]]
        then
            echo ""$line",transversion" >> 3_10.vcf
        else
            echo ""$line",substitution" >> 3_10.vcf   
        fi
    done < 2_10.vcf
sed '2d' 3_10.vcf  > 4_10.vcf
sed -i '1d' 4_10.vcf
echo "POS,REF,ALT,QUAL,DEPTH,REF_READS,ALT_READS,TYPE,GENE" > "$s"_SNPs_10.vcf
    while IFS= read -r line; 
    do
    pos=$(echo ""$line"" | sed  's/,.*$//');
    nc_region='TRUE'
        for c in "${!gene_csv[@]}"
        do
            if (( "$pos" >= "${start_csv[$c]}" && "$pos" <= "${end_csv[$c]}" ));
            then
                nc_region='FALSE'
                echo ""$line",""${gene_csv[$c]}""," >> "$s"_SNPs_10.vcf;
            fi
        done
        if [[ "$nc_region" == 'TRUE' ]];
        then 
            echo ""$line",-," >> "$s"_SNPs_10.vcf;
        fi
        nc_region='TRUE'
    done < 4_10.vcf
rm 1_10.vcf
rm 2_10.vcf
rm 3_10.vcf
rm 4_10.vcf
done

#VCF Formatting for 5% SNP read support
for s in "${samples[@]}"
do 
cd "$snp_dir"/"$s"/"$s"_Export/
sed '1,60d' "$snp_dir"/"$s"/"$s"_Export/bwa_paired_"$s"_sorted_freebayes_0.05_SNPs.vcf > 1_5.vcf
sed -e "s/AB\=.*:GL//g" -e "s/.\/.\://g" -e "s/\:/\,/1" -e "s/\:.*//g" -e "s/\s\+/,/g" -e "s/\,ID//g" -e "s/\,FILTER\,INFO\,FORMAT\,unknown/\,DEPTH\,REF\_READS\,ALT\_READS\,TYPE/g" -e "s/\,\.//g" -e "s/\#CHROM\,//g" -e "s/\,/\:/1" -e "s/.*\://g" -e "s/REF/POS\,REF/g" 1_5.vcf > 2_5.vcf
echo "POS,REF,ALT,QUAL,DEPTH,REF_READS,ALT_READS,TYPE" > 3_5.vcf
    while IFS= read -r line; 
    do
        if [[ "$line" =~ ",A," ]] && [[ "$line" =~ ",C," ]]
        then
           echo ""$line",transversion" >> 3_5.vcf
        elif [[ "$line" =~ ",A," ]] && [[ "$line" =~ ",G," ]]
        then
            echo ""$line",transition" >> 3_5.vcf
        elif [[ "$line" =~ ",A," ]] && [[ "$line" =~ ",T," ]]
        then
            echo ""$line",transversion" >> 3_5.vcf
        elif [[ "$line" =~ ",C," ]] && [[ "$line" =~ ",G," ]]
        then
            echo ""$line",transversion" >> 3_5.vcf
        elif [[ "$line" =~ ",C," ]] && [[ "$line" =~ ",T," ]]
        then
            echo ""$line",transition" >> 3_5.vcf
        elif [[ "$line" =~ ",G," ]] && [[ "$line" =~ ",T," ]]
        then
            echo ""$line",transversion" >> 3_5.vcf
        else
            echo ""$line",substitution" >> 3_5.vcf    
        fi
    done < 2_5.vcf
sed '2d' 3_5.vcf  > 4_5.vcf
sed -i '1d' 4_5.vcf
echo "POS,REF,ALT,QUAL,DEPTH,REF_READS,ALT_READS,TYPE,GENE" > "$s"_SNPs_5.vcf
    while IFS= read -r line; 
    do
    pos=$(echo ""$line"" | sed  's/,.*$//');
    nc_region='TRUE'
        for c in "${!gene_csv[@]}"
        do
            if (( "$pos" >= "${start_csv[$c]}" && "$pos" <= "${end_csv[$c]}" ));
            then
                nc_region='FALSE'
                echo ""$line",""${gene_csv[$c]}""," >> "$s"_SNPs_5.vcf;
            fi
        done
        if [[ "$nc_region" == 'TRUE' ]];
        then 
            echo ""$line",-," >> "$s"_SNPs_5.vcf;
        fi
        nc_region='TRUE'
    done < 4_5.vcf
rm 1_5.vcf
rm 2_5.vcf
rm 3_5.vcf
rm 4_5.vcf
done

#VCF Formatting for 1% SNP read support
for s in "${samples[@]}"
do
cd "$snp_dir"/"$s"/"$s"_Export/
sed '1,60d' "$snp_dir"/"$s"/"$s"_Export/bwa_paired_"$s"_sorted_freebayes_0.01_SNPs.vcf > 1_1.vcf
sed -e "s/AB\=.*:GL//g" -e "s/.\/.\://g" -e "s/\:/\,/1" -e "s/\:.*//g" -e "s/\s\+/,/g" -e "s/\,ID//g" -e "s/\,FILTER\,INFO\,FORMAT\,unknown/\,DEPTH\,REF\_READS\,ALT\_READS\,TYPE/g" -e "s/\,\.//g" -e "s/\#CHROM\,//g" -e "s/\,/\:/1" -e "s/.*\://g" -e "s/REF/POS\,REF/g" 1_1.vcf > 2_1.vcf
echo "POS,REF,ALT,QUAL,DEPTH,REF_READS,ALT_READS,TYPE,GENE" > 3_1.vcf
    while IFS= read -r line; 
    do
        if [[ "$line" =~ ",A," ]] && [[ "$line" =~ ",C," ]]
        then
           echo ""$line",transversion" >> 3_1.vcf
        elif [[ "$line" =~ ",A," ]] && [[ "$line" =~ ",G," ]]
        then
            echo ""$line",transition" >> 3_1.vcf
        elif [[ "$line" =~ ",A," ]] && [[ "$line" =~ ",T," ]]
        then
            echo ""$line",transversion" >> 3_1.vcf
        elif [[ "$line" =~ ",C," ]] && [[ "$line" =~ ",G," ]]
        then
            echo ""$line",transversion" >> 3_1.vcf
        elif [[ "$line" =~ ",C," ]] && [[ "$line" =~ ",T," ]]
        then
            echo ""$line",transition" >> 3_1.vcf
        elif [[ "$line" =~ ",G," ]] && [[ "$line" =~ ",T," ]]
        then
            echo ""$line",transversion" >> 3_1.vcf
        else
            echo ""$line",substitution" >> 3_1.vcf    
        fi
    done < 2_1.vcf
sed '2d' 3_1.vcf  > 4_1.vcf
sed -i '1d' 4_1.vcf
echo "POS,REF,ALT,QUAL,DEPTH,REF_READS,ALT_READS,TYPE,GENE" > "$s"_SNPs_1.vcf
    while IFS= read -r line; 
    do
    pos=$(echo ""$line"" | sed  's/,.*$//');
    nc_region='TRUE'
        for c in "${!gene_csv[@]}"
        do
            if (( "$pos" >= "${start_csv[$c]}" && "$pos" <= "${end_csv[$c]}" ));
            then
                nc_region='FALSE'
                echo ""$line",""${gene_csv[$c]}""," >> "$s"_SNPs_1.vcf;
            fi
        done
        if [[ "$nc_region" == 'TRUE' ]];
        then 
            echo ""$line",-," >> "$s"_SNPs_1.vcf;
        fi
        nc_region='TRUE'
    done < 4_1.vcf
rm 1_1.vcf
rm 2_1.vcf
rm 3_1.vcf
rm 4_1.vcf
done

#Concatenating the VCFs for 50% SNP read support
for s in "${samples[@]}"
do
cd "$snp_dir"/"$s"/"$s"_Export/ 
echo "POS,REF,ALT,QUAL,DEPTH,REF_READS,ALT_READS,TYPE,GENE, Start, End, Possible SNP Start (Direct), Possible SNP End (Direct), Possible SNP Start (Complement), Possible SNP End (Complement), AA Change" > 1_50.csv
sed '1d' "$analysis_dir"/results/"$s"/aa_changes/"$s"_aa_change_50.csv > working_50.csv
sed '1d' "$snp_dir"/"$s"/"$s"_Export/"$s"_SNPs_50.vcf > working_50.vcf
st_dir=( $(tail -n +1 working_50.csv  | cut -d ',' -f4) )
end_dir=( $(tail -n +1 working_50.csv  | cut -d ',' -f5) )
st_cmp=( $(tail -n +1 working_50.csv  | cut -d ',' -f6) )
end_cmp=( $(tail -n +1 working_50.csv  | cut -d ',' -f7) )
gene_csv=( $(tail -n +1 working_50.csv  | cut -d ',' -f1) )
wc -l < working_50.csv
    while IFS= read -r line1; 
    do
    lineadder='TRUE'
    pos=( $(echo ""$line1"" | sed  's/,.*$//') )
    pre_gene=( $(echo ""$line1"" | sed 's/,\([^,]*\)$/\1/') )
    gene_vcf=( $(echo ""$pre_gene"" | sed 's/^.*,//') )
        for c in "${!st_dir[@]}"
        do
            if (( "$pos" >= "${st_dir[$c]}" && "$pos" <= "${end_dir[$c]}" )) && [ ${gene_csv[$c]} = $gene_vcf ];
            then
                lineadder='FALSE';
                line2=$(grep -m 1 ""${st_dir[$c]}","${end_dir[$c]}"," working_50.csv | head -n1)
                echo ""$line1""$line2"" >> 1_50.csv;
                sed -i '0,/'"${st_dir[$c]}"','"${end_dir[$c]}"'/{//d}' working_50.csv 
                sed -i '0,/'""$pos""'/{//d}' working_50.vcf
                unset st_dir[$c]
                unset end_dir[$c]
                unset st_cmp[$c]
                unset end_cmp[$c]
                unset gene_csv[$c]
                break
            elif (( "$pos" >= "${st_cmp[$c]}" && "$pos" <= "${end_cmp[$c]}" )) && [ ${gene_csv[$c]} = $gene_vcf ];
            then
                lineadder='FALSE';
                line2=$(grep -m 1 ""${st_cmp[$c]}","${end_cmp[$c]}"," working_50.csv | head -n1)
                echo ""$line1""$line2"" >> 1_50.csv;
                sed -i '0,/'"${st_cmp[$c]}"','"${end_cmp[$c]}"'/{//d}' working_50.csv 
                sed -i '0,/'""$pos""'/{//d}' working_50.vcf
                unset st_dir[$c]
                unset end_dir[$c]
                unset st_cmp[$c]
                unset end_cmp[$c]
                unset gene_csv[$c]
                break
            fi
        done
        if [[ "$lineadder" == 'TRUE' ]];
        then
            echo ""$line1"-,-,-,-,-,-,-,-" >> 1_50.csv;
            sed -i '/'"$pos"'/d' working_50.vcf
        fi
    done < working_50.vcf
remove='10,11,12,13,14,15,16'
cut -d ',' -f$remove --complement 1_50.csv > 2_50.csv
awk -F, '{ print $1 "," $9 "," $2 "," $3 "," $4 "," $5 "," $6 "," $7 "," $8 "," $10}' 2_50.csv > 3_50.csv
awk -F, '{ print $1 "," $2 "," $6 "," $3 "," $7 "," $4 "," $8 "," $5 "," $9 "," $10}' 3_50.csv > "$s"_SNPs_50_final.csv
cp "$s"_SNPs_50_final.csv "$analysis_dir"/results/"$s"/
rm working_50.vcf
rm working_50.csv
rm 1_50.csv
rm 2_50.csv
rm 3_50.csv
done

#Concatenating the VCFs for 10% SNP read support
for s in "${samples[@]}"
do
cd "$snp_dir"/"$s"/"$s"_Export/ 
echo "POS,REF,ALT,QUAL,DEPTH,REF_READS,ALT_READS,TYPE,GENE, Start, End, Possible SNP Start (Direct), Possible SNP End (Direct), Possible SNP Start (Complement), Possible SNP End (Complement), AA Change" > 1_10.csv
sed '1d' "$analysis_dir"/results/"$s"/aa_changes/"$s"_aa_change_10.csv > working_10.csv
sed '1d' "$snp_dir"/"$s"/"$s"_Export/"$s"_SNPs_10.vcf > working_10.vcf
st_dir=( $(tail -n +1 working_10.csv  | cut -d ',' -f4) )
end_dir=( $(tail -n +1 working_10.csv  | cut -d ',' -f5) )
st_cmp=( $(tail -n +1 working_10.csv  | cut -d ',' -f6) )
end_cmp=( $(tail -n +1 working_10.csv  | cut -d ',' -f7) )
gene_csv=( $(tail -n +1 working_10.csv  | cut -d ',' -f1) )
wc -l < working_10.csv
    while IFS= read -r line1; 
    do
    lineadder='TRUE'
    pos=( $(echo ""$line1"" | sed  's/,.*$//') )
    pre_gene=( $(echo ""$line1"" | sed 's/,\([^,]*\)$/\1/') )
    gene_vcf=( $(echo ""$pre_gene"" | sed 's/^.*,//') )
        for c in "${!st_dir[@]}"
        do
            if (( "$pos" >= "${st_dir[$c]}" && "$pos" <= "${end_dir[$c]}" )) && [ "${gene_csv[$c]}" = "$gene_vcf" ];
            then
                lineadder='FALSE';
                line2=$(grep -m 1 ""${st_dir[$c]}","${end_dir[$c]}"," working_10.csv | head -n1)
                echo ""$line1""$line2"" >> 1_10.csv;
                sed -i '0,/'"${st_dir[$c]}"','"${end_dir[$c]}"'/{//d}' working_10.csv 
                sed -i '0,/'""$pos""'/{//d}' working_10.vcf
                unset st_dir[$c]
                unset end_dir[$c]
                unset st_cmp[$c]
                unset end_cmp[$c]
                unset gene_csv[$c]
                break
            elif (( "$pos" >= "${st_cmp[$c]}" && "$pos" <= "${end_cmp[$c]}" )) && [ "${gene_csv[$c]}" = "$gene_vcf" ];
            then
                lineadder='FALSE';
                line2=$(grep -m 1 ""${st_cmp[$c]}","${end_cmp[$c]}"," working_10.csv | head -n1)
                echo ""$line1""$line2"" >> 1_10.csv;
                sed -i '0,/'"${st_cmp[$c]}"','"${end_cmp[$c]}"'/{//d}' working_10.csv 
                sed -i '0,/'""$pos""'/{//d}' working_10.vcf
                unset st_dir[$c]
                unset end_dir[$c]
                unset st_cmp[$c]
                unset end_cmp[$c]
                unset gene_csv[$c]
                break
            fi
        done
        if [[ "$lineadder" == 'TRUE' ]];
        then
            echo ""$line1"-,-,-,-,-,-,-,-" >> 1_10.csv;
            sed -i '/'"$pos"'/d' working_10.vcf
        fi
    done < working_10.vcf
remove='10,11,12,13,14,15,16'
cut -d ',' -f$remove --complement 1_10.csv > 2_10.csv
awk -F, '{ print $1 "," $9 "," $2 "," $3 "," $4 "," $5 "," $6 "," $7 "," $8 "," $10}' 2_10.csv > 3_10.csv
awk -F, '{ print $1 "," $2 "," $6 "," $3 "," $7 "," $4 "," $8 "," $5 "," $9 "," $10}' 3_10.csv > "$s"_SNPs_10_final.csv
cp "$s"_SNPs_10_final.csv "$analysis_dir"/results/"$s"/
rm working_10.vcf
rm working_10.csv
rm 1_10.csv
rm 2_10.csv
rm 3_10.csv
done

#Concatenating the VCFs for 5% SNP read support
for s in "${samples[@]}"
do
cd "$snp_dir"/"$s"/"$s"_Export/ 
echo "POS,REF,ALT,QUAL,DEPTH,REF_READS,ALT_READS,TYPE,GENE, Start, End, Possible SNP Start (Direct), Possible SNP End (Direct), Possible SNP Start (Complement), Possible SNP End (Complement), AA Change" > 1_5.csv
sed '1d' "$analysis_dir"/results/"$s"/aa_changes/"$s"_aa_change_5.csv > working_5.csv
sed '1d' "$snp_dir"/"$s"/"$s"_Export/"$s"_SNPs_5.vcf > working_5.vcf
st_dir=( $(tail -n +1 working_5.csv  | cut -d ',' -f4) )
end_dir=( $(tail -n +1 working_5.csv  | cut -d ',' -f5) )
st_cmp=( $(tail -n +1 working_5.csv  | cut -d ',' -f6) )
end_cmp=( $(tail -n +1 working_5.csv  | cut -d ',' -f7) )
gene_csv=( $(tail -n +1 working_5.csv  | cut -d ',' -f1) )
wc -l < working_5.csv
    while IFS= read -r line1; 
    do
    lineadder='TRUE'
    pos=( $(echo ""$line1"" | sed  's/,.*$//') )
    pre_gene=( $(echo ""$line1"" | sed 's/,\([^,]*\)$/\1/') )
    gene_vcf=( $(echo ""$pre_gene"" | sed 's/^.*,//') )
        for c in "${!st_dir[@]}"
        do
            if (( "$pos" >= "${st_dir[$c]}" && "$pos" <= "${end_dir[$c]}" )) && [ ${gene_csv[$c]} = $gene_vcf ];
            then
                lineadder='FALSE';
                line2=$(grep -m 1 ""${st_dir[$c]}","${end_dir[$c]}"," working_5.csv | head -n1)
                echo ""$line1""$line2"" >> 1_5.csv;
                sed -i '0,/'"${st_dir[$c]}"','"${end_dir[$c]}"'/{//d}' working_5.csv 
                sed -i '0,/'""$pos""'/{//d}' working_5.vcf
                unset st_dir[$c]
                unset end_dir[$c]
                unset st_cmp[$c]
                unset end_cmp[$c]
                unset gene_csv[$c]
                break
            elif (( "$pos" >= "${st_cmp[$c]}" && "$pos" <= "${end_cmp[$c]}" )) && [ ${gene_csv[$c]} = $gene_vcf ];
            then
                lineadder='FALSE';
                line2=$(grep -m 1 ""${st_cmp[$c]}","${end_cmp[$c]}"," working_5.csv | head -n1)
                echo ""$line1""$line2"" >> 1_5.csv;
                sed -i '0,/'"${st_cmp[$c]}"','"${end_cmp[$c]}"'/{//d}' working_5.csv 
                sed -i '0,/'""$pos""'/{//d}' working_5.vcf
                unset st_dir[$c]
                unset end_dir[$c]
                unset st_cmp[$c]
                unset end_cmp[$c]
                unset gene_csv[$c]
                break
            fi
        done
        if [[ "$lineadder" == 'TRUE' ]];
        then
            echo ""$line1"-,-,-,-,-,-,-,-" >> 1_5.csv;
            sed -i '/'"$pos"'/d' working_5.vcf
        fi
    done < working_5.vcf
remove='10,11,12,13,14,15,16'
cut -d ',' -f$remove --complement 1_5.csv > 2_5.csv
awk -F, '{ print $1 "," $9 "," $2 "," $3 "," $4 "," $5 "," $6 "," $7 "," $8 "," $10}' 2_5.csv > 3_5.csv
awk -F, '{ print $1 "," $2 "," $6 "," $3 "," $7 "," $4 "," $8 "," $5 "," $9 "," $10}' 3_5.csv > "$s"_SNPs_5_final.csv
cp "$s"_SNPs_5_final.csv "$analysis_dir"/results/"$s"/
rm working_5.vcf
rm working_5.csv
rm 1_5.csv
rm 2_5.csv
rm 3_5.csv
done

#Concatenating the VCFs for 1% SNP read support
for s in "${samples[@]}"
do
cd "$snp_dir"/"$s"/"$s"_Export/ 
echo "POS,REF,ALT,QUAL,DEPTH,REF_READS,ALT_READS,TYPE,GENE, Start, End, Possible SNP Start (Direct), Possible SNP End (Direct), Possible SNP Start (Complement), Possible SNP End (Complement), AA Change" > 1_1.csv
sed '1d' "$analysis_dir"/results/"$s"/aa_changes/"$s"_aa_change_1.csv > working_1.csv
sed '1d' "$snp_dir"/"$s"/"$s"_Export/"$s"_SNPs_1.vcf > working_1.vcf
st_dir=( $(tail -n +1 working_1.csv  | cut -d ',' -f4) )
end_dir=( $(tail -n +1 working_1.csv  | cut -d ',' -f5) )
st_cmp=( $(tail -n +1 working_1.csv  | cut -d ',' -f6) )
end_cmp=( $(tail -n +1 working_1.csv  | cut -d ',' -f7) )
gene_csv=( $(tail -n +1 working_1.csv  | cut -d ',' -f1) )
wc -l < working_1.csv
    while IFS= read -r line1; 
    do
    lineadder='TRUE'
    pos=( $(echo ""$line1"" | sed  's/,.*$//') )
    pre_gene=( $(echo ""$line1"" | sed 's/,\([^,]*\)$/\1/') )
    gene_vcf=( $(echo ""$pre_gene"" | sed 's/^.*,//') )
        for c in "${!st_dir[@]}"
        do
            if (( "$pos" >= "${st_dir[$c]}" && "$pos" <= "${end_dir[$c]}" )) && [ "${gene_csv[$c]}" = "$gene_vcf" ];
            then
                lineadder='FALSE';
                line2=$(grep -m 1 ""${st_dir[$c]}","${end_dir[$c]}"," working_1.csv | head -n1)
                echo ""$line1""$line2"" >> 1_1.csv;
                sed -i '0,/'"${st_dir[$c]}"','"${end_dir[$c]}"'/{//d}' working_1.csv 
                sed -i '0,/'""$pos""'/{//d}' working_1.vcf
                unset st_dir[$c]
                unset end_dir[$c]
                unset st_cmp[$c]
                unset end_cmp[$c]
                unset gene_csv[$c]
                break
            elif (( "$pos" >= "${st_cmp[$c]}" && "$pos" <= "${end_cmp[$c]}" )) && [ "${gene_csv[$c]}" = "$gene_vcf" ];
            then
                lineadder='FALSE';
                line2=$(grep -m 1 ""${st_cmp[$c]}","${end_cmp[$c]}"," working_1.csv | head -n1)
                echo ""$line1""$line2"" >> 1_1.csv;
                sed -i '0,/'"${st_cmp[$c]}"','"${end_cmp[$c]}"'/{//d}' working_1.csv 
                sed -i '0,/'""$pos""'/{//d}' working_1.vcf
                unset st_dir[$c]
                unset end_dir[$c]
                unset st_cmp[$c]
                unset end_cmp[$c]
                unset gene_csv[$c]
                break
            fi
        done
        if [[ "$lineadder" == 'TRUE' ]];
        then
            echo ""$line1"-,-,-,-,-,-,-,-" >> 1_1.csv;
            sed -i '/'"$pos"'/d' working_1.vcf
        fi
    done < working_1.vcf
remove='10,11,12,13,14,15,16'
cut -d ',' -f$remove --complement 1_1.csv > 2_1.csv
awk -F, '{ print $1 "," $9 "," $2 "," $3 "," $4 "," $5 "," $6 "," $7 "," $8 "," $10}' 2_1.csv > 3_1.csv
awk -F, '{ print $1 "," $2 "," $6 "," $3 "," $7 "," $4 "," $8 "," $5 "," $9 "," $10}' 3_1.csv > "$s"_SNPs_1_final.csv
cp "$s"_SNPs_1_final.csv "$analysis_dir"/results/"$s"/
rm working_1.vcf
rm working_1.csv
rm 1_1.csv
rm 2_1.csv
rm 3_1.csv
done

## Protein Function Prediction
Predicting possible alterations in protein function caused by AA changes. 
#### Provean
For more information on the command line PROVEAN tool see [here](http://provean.jcvi.org/downloads/README). 

In [ ]:
#PROVEAN predictions for 50% SNP read support
#for s in "${samples[@]}"
#do
#cd "$output_dir"/"$s"_50/aa_changes/
#mkdir PROVEAN
#cd "$output_dir"/"$s"_50/aa_changes/PROVEAN/
#    for gene in "${gene_csv[@]}"
#    do
#    /home/fuberlin/Programs/provean-1.1.5/scripts/provean.sh -q "$output_dir"/"$refname"/gene-"$gene"_ref.fasta -v "$output_dir"/"$s"_10/aa_changes/"$gene".var > "$gene"_PROVEAN.txt
#    done
#done
#create a csv with the change, score and classification (if statement with neutral and deleterious)
#use csvtool to append the scores and classification of the aa changes into the main csv

Deleting unnecessary files and folders:

In [ ]:
#Housekeeping, removing all unecessary files

#rm -r "$output_dir"